## Machine Translation Project

## Introduction
In this notebook, we will build a deep neural network that functions as part of an end-to-end machine translation pipeline. The completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - convert text to sequence of integers.
- **Models** - Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations.
- **Prediction** - Run the model on English text.

In [ ]:
#imports
import os
import collections
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, LSTM, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional ,Dropout
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

In [ ]:
#load dataset
def load_data(path):
    input_file = os.path.join(path)
    with open(input_file, "r") as f:
        data = f.read()

    return data.split('\n')

### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [ ]:
#load English Data
english_sentences = load_data('data/small_vocab_en')
#load French Data
french_sentences = load_data('data/small_vocab_fr')

### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [ ]:
#visualize data
for text in range(2):
  print('Englist text {}: {}'.format(text+1, english_sentences[text]))
  print('French text {}: {}'.format(text+1, french_sentences[text]))


Englist text 1: new jersey is sometimes quiet during autumn , and it is snowy in april .
French text 1: new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
Englist text 2: the united states is usually chilly during july , and it is usually freezing in november .
French text 2: les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


In [ ]:
#total no of unique words in each vocabualary
english_word_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_word_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English word.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique Englist words.'.format(len(english_word_counter)))
print('10 Most common words in English dataset:')
print('"' + '" "'. join(list(zip(*english_word_counter.most_common(10)))[0])+'"')
print("")
print('{} French word.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_word_counter)))
print('10 Most common words in French dataset:')
print('"' + '" "'. join(list(zip(*french_word_counter.most_common(10)))[0])+'"')

1823250 English word.
227 unique Englist words.
10 Most common words in English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French word.
355 unique French words.
10 Most common words in French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


## Preprocess
we need to convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

### Tokenize
we will Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. 

In [ ]:
def tokenize(sentences):
  """
  Tokenize sentences
  :param sentence: List of sentences to be tokenized
  :return: Tuple of tokenized sentences and tokenizer
  """
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(sentences)
  text_tokenized = tokenizer.texts_to_sequences(sentences)

  return text_tokenized, tokenizer

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding
we need to make sure that all the sequences are of same length by adding padding to the end of each sequence

In [ ]:
def pad(sentences, length=None):
  """
  pad sentences
  :param sentences: List of sentences
  :parem length: Lenght to pad the sequence to. if None use the length of the longest sequence
  :return: padded numpy array of sequences
  """
  if length is None:
    length=max([len(sentence) for sentence in sentences])

  return pad_sequences(sentences, maxlen=length, padding='post')

tests.test_pad(pad)

#pad Tokenized output
test_pad = pad(text_tokenized)
for text, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
  print('Sequence {} in x'.format(text+1))
  print('   Input: {}'.format(np.array(token_sent)))
  print('   Output: {}'.format(pad_sent))

Sequence 1 in x
   Input: [1 2 4 5 6 7 1 8 9]
   Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
   Input: [10 11 12  2 13 14 15 16  3 17]
   Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
   Input: [18 19  3 20 21]
   Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline


In [ ]:
def preprocess(x, y):
  """
  preprocess x and y
  :param x: Feature List of sentences
  :param y: Label List of sentences
  :return: Tuple of (Preprocessed x, Preprocessed y, x_tokenizer, y_tokenizer)
  """
  preprocess_x, x_tokenizer = tokenize(x)
  preprocess_y, y_tokenizer = tokenize(y)

  preprocess_x = pad(preprocess_x)
  preprocess_y = pad(preprocess_y)

  #keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
  preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

  return preprocess_x, preprocess_y, x_tokenizer, y_tokenizer

preprocessed_english_sentences, preprocessed_french_sentences, english_tokenizer, french_tokenizer =\
  preprocess(english_sentences, french_sentences)


max_english_sequence_length = preprocessed_english_sentences.shape[1]
max_french_sequence_length = preprocessed_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


In [ ]:
def logits_to_text(logits, tokenizer):
  """
  Turn logits from Neural Network into text using the tokenizer
  :param logits: Logits from Neural Network
  :param tokenizer: keras Tokenizer fit on the labels
  :return: String that represents the text of logits
  """
  index_to_words = {id:word for word, id in tokenizer.word_index.items()}
  index_to_words[0]='<PAD>'

  return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

  
print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN
A basic RNN model is a good baseline for sequence data. 

In [ ]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
   
    # TODO: Build the layers
    learning_rate = 1e-2
    input_seq = Input(input_shape[1:])
    rnn = LSTM(64, return_sequences=True)(input_seq)
    dropout = Dropout(0.2)(rnn)
    # we change `plaintext_vocab_size` to `french_vocab_size` as our ouput is now French sentences intead of plain words
    logits = TimeDistributed(Dense(french_vocab_size))(dropout)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])

    
    return model

# tests.test_simple_model(simple_model)


#reshape input
tmp_x = pad(preprocessed_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preprocessed_french_sentences.shape[-2], 1))

#Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size
)

simple_rnn_model.fit(tmp_x, preprocessed_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)

#print predictions
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/10
110288/110288 [==============================] - 8s 74us/step - loss: 2.3451 - accuracy: 0.4917 - val_loss: nan - val_accuracy: 0.5742
Epoch 2/10
110288/110288 [==============================] - 6s 57us/step - loss: 1.6524 - accuracy: 0.5791 - val_loss: nan - val_accuracy: 0.6168
Epoch 3/10
110288/110288 [==============================] - 6s 58us/step - loss: 1.4886 - accuracy: 0.6039 - val_loss: nan - val_accuracy: 0.6284
Epoch 4/10
110288/110288 [==============================] - 6s 56us/step - loss: 1.3969 - accuracy: 0.6177 - val_loss: nan - val_accuracy: 0.6447
Epoch 5/10
110288/110288 [==============================] - 6s 54us/step - loss: 1.3360 - accuracy: 0.6271 - val_loss: nan - val_accuracy: 0.6502
Epoch 6/10
110288/110288 [==============================] - 6s 55us/step - loss: 1.2968 - accuracy: 0.6327 - val_loss: nan - val_accuracy: 0.6543
Epoch 7/10
110288/110288 [==============================] - 6s 55us/step 

### Model 2: Embedding (IMPLEMENTATION)
In the simple model we have used ids to represent words, but there's a better representation of a word. This is called word embeddings. An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, we willcreate a RNN model using embedding.

In [ ]:
def embedding_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  """
  Build and train a RNN model using word embedding on x and y
  :param input_shape: Tuple of input shape
  :param output_sequence_length: length og output sequence
  :param english_vocab_size: Number of unique English words in the dataset
  :param french_vocab_size: Number of unique French words in dataset
  :return: keras model built, but not trained
  """
  
  learning_rate = 1e-2
  
  input_seq = Input(input_shape[1:])
  
  embed_layer = Embedding(english_vocab_size, 64, input_length=output_sequence_length)(input_seq)

  rnn = LSTM(64, return_sequences=True)(embed_layer)

  dropout = Dropout(0.2)(rnn)

  logits = TimeDistributed(Dense(french_vocab_size))(dropout)

  model = Model(input_seq, Activation('softmax')(logits))

  model.compile(loss=sparse_categorical_crossentropy,
                optimizer=Adam(learning_rate),
                metrics=['accuracy'])
  
  return model

# tests.test_embed_model(embedding_model)

#reshaping input
tmp_x = pad(preprocessed_english_sentences, max_french_sequence_length)
# tmp_x = tmp_x.reshape((-1, preprocessed_french_sentences.shape[-2], 1))

#train the neural network
embedding_rnn_model = embedding_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
embedding_rnn_model.fit(tmp_x, preprocessed_french_sentences, batch_size=512, epochs=64, validation_split=0.2)

print(logits_to_text(embedding_rnn_model.predict(tmp_x[:1])[0] , french_tokenizer))


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/64
110288/110288 [==============================] - 18s 167us/step - loss: 1.7954 - accuracy: 0.6056 - val_loss: nan - val_accuracy: 0.7819
Epoch 2/64
110288/110288 [==============================] - 16s 146us/step - loss: 0.6699 - accuracy: 0.8070 - val_loss: nan - val_accuracy: 0.8605
Epoch 3/64
110288/110288 [==============================] - 16s 144us/step - loss: 0.4885 - accuracy: 0.8545 - val_loss: nan - val_accuracy: 0.8880
Epoch 4/64
110288/110288 [==============================] - 15s 138us/step - loss: 0.4150 - accuracy: 0.8763 - val_loss: nan - val_accuracy: 0.9006
Epoch 5/64
110288/110288 [==============================] - 16s 144us/step - loss: 0.3732 - accuracy: 0.8881 - val_loss: nan - val_accuracy: 0.9051
Epoch 6/64
110288/110288 [==============================] - 16s 141us/step - loss: 0.3469 - accuracy: 0.8957 - val_loss: nan - val_accuracy: 0.9119
Epoch 7/64
110288/110288 [==============================] - 1

### Model 3: Bidirectional RNNs 
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [ ]:
def bidrectional_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
  """
  Build and train a bidirectional RNN model on x and y
  :param input_shape: Tuple of input shape
  :param output_sequence_length: Length of output sequence
  :param english_vocab_size: Number of unique English words in the dataset
  :param french_vocab_size: Number of unique French words in the dataset
  :return: Keras model built, but not trained
  """

  learning_rate = 1e-2

  input_seq = Input(input_shape[1:])

  rnn = Bidirectional(LSTM(64, return_sequences=True))(input_seq)

  dropout = Dropout(0.2)(rnn)

  logits = TimeDistributed(Dense(french_vocab_size))(dropout)

  model = Model(input_seq, Activation('softmax')(logits))

  model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
  return model

# Reshaping the input to work with a basic RNN
tmp_x = pad(preprocessed_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preprocessed_french_sentences.shape[-2], 1))

# Train the neural network
bidrectional_rnn_model = bidrectional_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
bidrectional_rnn_model.fit(tmp_x, preprocessed_french_sentences, batch_size=512, epochs=64, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(bidrectional_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/64
110288/110288 [==============================] - 28s 253us/step - loss: 1.6933 - accuracy: 0.5827 - val_loss: nan - val_accuracy: 0.6481
Epoch 2/64
110288/110288 [==============================] - 27s 242us/step - loss: 1.2304 - accuracy: 0.6461 - val_loss: nan - val_accuracy: 0.6700
Epoch 3/64
110288/110288 [==============================] - 26s 238us/step - loss: 1.1412 - accuracy: 0.6619 - val_loss: nan - val_accuracy: 0.6851
Epoch 4/64
110288/110288 [==============================] - 26s 238us/step - loss: 1.0869 - accuracy: 0.6703 - val_loss: nan - val_accuracy: 0.6910
Epoch 5/64
110288/110288 [==============================] - 27s 241us/step - loss: 1.0497 - accuracy: 0.6761 - val_loss: nan - val_accuracy: 0.6962
Epoch 6/64
110288/110288 [==============================] - 26s 235us/step - loss: 1.0217 - accuracy: 0.6800 - val_loss: nan - val_accuracy: 0.6986
Epoch 7/64
110288/110288 [==============================] - 2

### Model 4: Encoder-Decoder
This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence. The decoder takes this matrix as input and predicts the translation as output.

In [ ]:
from keras.layers import RepeatVector

def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    

    learning_rate = 0.03

    #Encoder
    encoder_inputs = Input(shape=input_shape[1:])
    encoder_gru = GRU(output_sequence_length)(encoder_inputs)
    encoder_outputs = Dense(64, activation='relu')(encoder_gru)
    
    #Decoder
    decoder_inputs = RepeatVector(output_sequence_length)(encoder_outputs)
    decoder_gru = GRU(64, return_sequences=True)(decoder_inputs)
    output_layer = TimeDistributed(Dense(french_vocab_size, activation='softmax'))
    outputs = output_layer(decoder_gru)

    #Create Model from parameters defined above
    model = Model(inputs=encoder_inputs, outputs=outputs)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


#reshape input
tmp_x = pad(preprocessed_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preprocessed_french_sentences.shape[-2], 1))

# Train the neural network
encdec_rnn_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
encdec_rnn_model.fit(tmp_x, preprocessed_french_sentences, batch_size=512, epochs=64, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(encdec_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 110288 samples, validate on 27573 samples
Epoch 1/64
110288/110288 [==============================] - 23s 205us/step - loss: 2.3537 - accuracy: 0.4879 - val_loss: nan - val_accuracy: 0.5356
Epoch 2/64
110288/110288 [==============================] - 21s 186us/step - loss: 1.6896 - accuracy: 0.5542 - val_loss: nan - val_accuracy: 0.5664
Epoch 3/64
110288/110288 [==============================] - 20s 185us/step - loss: 1.5182 - accuracy: 0.5870 - val_loss: nan - val_accuracy: 0.6119
Epoch 4/64
110288/110288 [==============================] - 21s 194us/step - loss: 1.4224 - accuracy: 0.6068 - val_loss: nan - val_accuracy: 0.6144
Epoch 5/64
110288/110288 [==============================] - 21s 188us/step - loss: 1.3816 - accuracy: 0.6129 - val_loss: nan - val_accuracy: 0.6064
Epoch 6/64
110288/110288 [==============================] - 21s 188us/step - loss: 1.3888 - accuracy: 0.6096 - val_loss: nan - val_accuracy: 0.6105
Epoch 7/64
110288/110288 [==============================] - 2

### Model 5: Custom 
Use everything from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [ ]:
def model_final_GRU(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-2
    
    input_seq = Input(input_shape[1:])

    embed_layer = Embedding(english_vocab_size , 64 , input_length = output_sequence_length)(input_seq)
    
    rnn = Bidirectional(GRU(64, return_sequences=True))(embed_layer)
    
    dropout = Dropout(0.2)(rnn)
    
    logits = TimeDistributed(Dense(french_vocab_size))(dropout)
    
 
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    
    
    return model



print('Final Model Loaded')
# TODO: Train the final model

tmp_x = pad(preprocessed_english_sentences, max_french_sequence_length)
#tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

# Train the neural network
final_gru_rnn_model = model_final_GRU(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
final_gru_rnn_model.fit(tmp_x, preprocessed_french_sentences, batch_size=512, epochs=48, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(final_gru_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Final Model Loaded


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/48
110288/110288 [==============================] - 23s 210us/step - loss: 1.1498 - accuracy: 0.7211 - val_loss: nan - val_accuracy: 0.8767
Epoch 2/48
110288/110288 [==============================] - 22s 202us/step - loss: 0.3666 - accuracy: 0.8889 - val_loss: nan - val_accuracy: 0.9279
Epoch 3/48
110288/110288 [==============================] - 23s 208us/step - loss: 0.2712 - accuracy: 0.9201 - val_loss: nan - val_accuracy: 0.9440
Epoch 4/48
110288/110288 [==============================] - 22s 202us/step - loss: 0.2335 - accuracy: 0.9325 - val_loss: nan - val_accuracy: 0.9500
Epoch 5/48
110288/110288 [==============================] - 22s 201us/step - loss: 0.2079 - accuracy: 0.9403 - val_loss: nan - val_accuracy: 0.9539
Epoch 6/48
110288/110288 [==============================] - 23s 213us/step - loss: 0.1963 - accuracy: 0.9440 - val_loss: nan - val_accuracy: 0.9574
Epoch 7/48
110288/110288 [==============================] - 2

## Prediction

In [ ]:
def final_predictions_GRU(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final 
    max_french_sequence_length = y.shape[1]
    english_vocab_size = len(x_tk.word_index)
    french_vocab_size = len(y_tk.word_index)

    # Pad the input
    x = pad(x, max_french_sequence_length)

    # Train
    model = model_final_GRU(
        x.shape,
        max_french_sequence_length,
        english_vocab_size,
        french_vocab_size)

    model.fit(x, y, batch_size=512, epochs=80, validation_split=0.2)
    
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions_GRU(preprocessed_english_sentences, preprocessed_french_sentences, english_tokenizer, french_tokenizer)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 110288 samples, validate on 27573 samples
Epoch 1/80
110288/110288 [==============================] - 23s 208us/step - loss: 1.1390 - accuracy: 0.7199 - val_loss: nan - val_accuracy: 0.8709
Epoch 2/80
110288/110288 [==============================] - 23s 210us/step - loss: 0.3689 - accuracy: 0.8857 - val_loss: nan - val_accuracy: 0.9271
Epoch 3/80
110288/110288 [==============================] - 22s 202us/step - loss: 0.2660 - accuracy: 0.9212 - val_loss: nan - val_accuracy: 0.9434
Epoch 4/80
110288/110288 [==============================] - 22s 201us/step - loss: 0.2273 - accuracy: 0.9338 - val_loss: nan - val_accuracy: 0.9512
Epoch 5/80
110288/110288 [==============================] - 22s 202us/step - loss: 0.2061 - accuracy: 0.9407 - val_loss: nan - val_accuracy: 0.9561
Epoch 6/80
110288/110288 [==============================] - 23s 209us/step - loss: 0.1933 - accuracy: 0.9449 - val_loss: nan - val_accuracy: 0.9577
Epoch 7/80
110288/110288 [==============================] - 2